# NB03: Environmental Correlation of PHB Pathway Distribution

**Purpose**: Test whether species with PHB pathways are enriched in environments with temporally variable carbon availability (the feast/famine hypothesis).

**Requires**: BERDL JupyterHub (Spark session)

**Inputs**:
- `data/phb_by_taxonomy.tsv` from NB02

**Outputs**:
- `data/species_environment.tsv` — species-level environment classifications
- `data/phb_embedding_analysis.tsv` — AlphaEarth embedding analysis
- `figures/phb_by_environment.png` — PHB prevalence by environment type
- `figures/embedding_variance_phb.png` — Environmental breadth (embedding variance) vs PHB status

## Environmental Scoring Strategy

Three independent environment axes:
1. **NCBI labels**: isolation_source, env_broad_scale from `ncbi_env` (EAV format)
2. **AlphaEarth embeddings**: 64-dim environmental embeddings (28% genome coverage)
3. **Environment classification**: Categorize species as soil/marine/freshwater/host-associated/etc.

In [ ]:
spark = get_spark_session()

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

PROJECT_DIR = os.path.expanduser('~/BERIL-research-observatory/projects/phb_granule_ecology')
DATA_DIR = os.path.join(PROJECT_DIR, 'data')
FIG_DIR = os.path.join(PROJECT_DIR, 'figures')

# Load NB02 results
tax_phb = pd.read_csv(os.path.join(DATA_DIR, 'phb_by_taxonomy.tsv'), sep='\t')
print(f'Loaded {len(tax_phb):,} species')

## Part 1: NCBI Environment Metadata

Extract isolation_source and environmental labels for genomes, aggregate to species level.

In [ ]:
# Extract environment metadata for all genomes
# ncbi_env is EAV format — pivot to get isolation_source, env_broad_scale, host
env_data = spark.sql("""
    SELECT g.genome_id, g.gtdb_species_clade_id,
           MAX(CASE WHEN ne.harmonized_name = 'isolation_source' THEN ne.content END) as isolation_source,
           MAX(CASE WHEN ne.harmonized_name = 'env_broad_scale' THEN ne.content END) as env_broad_scale,
           MAX(CASE WHEN ne.harmonized_name = 'env_local_scale' THEN ne.content END) as env_local_scale,
           MAX(CASE WHEN ne.harmonized_name = 'env_medium' THEN ne.content END) as env_medium,
           MAX(CASE WHEN ne.harmonized_name = 'host' THEN ne.content END) as host,
           MAX(CASE WHEN ne.harmonized_name = 'geo_loc_name' THEN ne.content END) as geo_loc_name
    FROM kbase_ke_pangenome.genome g
    JOIN kbase_ke_pangenome.ncbi_env ne ON g.ncbi_biosample_id = ne.accession
    GROUP BY g.genome_id, g.gtdb_species_clade_id
""").toPandas()

print(f'Genomes with environment metadata: {len(env_data):,}')
print(f'Species represented: {env_data["gtdb_species_clade_id"].nunique():,}')
print(f'\nField coverage:')
for col in ['isolation_source', 'env_broad_scale', 'env_local_scale', 'host', 'geo_loc_name']:
    n = env_data[col].notna().sum()
    print(f'  {col}: {n:,} ({n/len(env_data)*100:.1f}%)')

In [ ]:
# Classify environments into broad categories
# Based on isolation_source keywords (following env_embedding_explorer approach)

def classify_environment(row):
    """Classify genome environment from NCBI metadata."""
    source = str(row.get('isolation_source', '')).lower()
    host = str(row.get('host', '')).lower()
    env_broad = str(row.get('env_broad_scale', '')).lower()
    
    # Human/clinical
    if any(kw in source for kw in ['blood', 'sputum', 'urine', 'wound', 'clinical',
                                     'patient', 'hospital', 'human']):
        return 'human_clinical'
    if 'homo sapiens' in host or 'human' in host:
        return 'human_associated'
    
    # Animal-associated
    if any(kw in source for kw in ['animal', 'bovine', 'chicken', 'pig', 'cattle',
                                     'poultry', 'feces', 'gut', 'intestin']):
        return 'animal_associated'
    
    # Soil/rhizosphere
    if any(kw in source for kw in ['soil', 'rhizosphere', 'root', 'compost', 'peat']):
        return 'soil'
    
    # Plant-associated
    if any(kw in source for kw in ['plant', 'leaf', 'stem', 'flower', 'seed', 'phyllosphere']):
        return 'plant_associated'
    
    # Freshwater
    if any(kw in source for kw in ['freshwater', 'lake', 'river', 'pond', 'stream',
                                     'groundwater', 'spring']):
        return 'freshwater'
    
    # Marine
    if any(kw in source for kw in ['marine', 'ocean', 'sea', 'seawater', 'coastal',
                                     'estuarine', 'estuary', 'tidal', 'coral']):
        return 'marine'
    
    # Wastewater/engineered
    if any(kw in source for kw in ['wastewater', 'sewage', 'activated sludge', 'bioreactor',
                                     'ferment']):
        return 'wastewater_engineered'
    
    # Sediment
    if any(kw in source for kw in ['sediment', 'mud', 'silt']):
        return 'sediment'
    
    # Food
    if any(kw in source for kw in ['food', 'milk', 'cheese', 'meat', 'fish']):
        return 'food'
    
    return 'other_unknown'

env_data['env_category'] = env_data.apply(classify_environment, axis=1)
print('Environment classification:')
print(env_data['env_category'].value_counts())

In [ ]:
# Classify environments by expected temporal variability
# Key axis for the feast/famine hypothesis

VARIABILITY_SCORES = {
    'soil': 'high',               # Highly variable C (root exudates, rainfall, seasons)
    'plant_associated': 'high',    # Variable C from plant metabolism, diel cycles
    'wastewater_engineered': 'high',  # Engineered feast/famine cycles
    'freshwater': 'moderate',      # Seasonal, but less extreme
    'sediment': 'moderate',        # Tidal/seasonal, redox gradients
    'marine': 'low',              # Relatively stable in open ocean
    'human_clinical': 'low',       # Stable host environment
    'human_associated': 'low',     # Stable host
    'animal_associated': 'low',    # Stable host
    'food': 'moderate',           # Processing creates variability
    'other_unknown': 'unknown',
}

env_data['variability'] = env_data['env_category'].map(VARIABILITY_SCORES)
print('Environmental variability classification:')
print(env_data['variability'].value_counts())

In [ ]:
# Aggregate environment to species level
# A species can have genomes from multiple environments — take the most common
species_env = env_data.groupby('gtdb_species_clade_id').agg(
    n_genomes_with_env=('genome_id', 'count'),
    primary_env=('env_category', lambda x: x.value_counts().index[0]),
    n_env_categories=('env_category', 'nunique'),
    primary_variability=('variability', lambda x: x.value_counts().index[0]),
    pct_high_var=('variability', lambda x: (x == 'high').mean() * 100),
    pct_low_var=('variability', lambda x: (x == 'low').mean() * 100),
).reset_index()

print(f'Species with environment data: {len(species_env):,}')
print(f'\nPrimary environment distribution:')
print(species_env['primary_env'].value_counts())

In [ ]:
# Merge with PHB status
env_phb = species_env.merge(tax_phb[['gtdb_species_clade_id', 'phb_status', 'has_phaC', 
                                      'has_complete_pathway', 'gtdb_phylum']],
                             on='gtdb_species_clade_id', how='inner')

# PHB prevalence by environment category
env_phb_stats = env_phb.groupby('primary_env').agg(
    n_species=('gtdb_species_clade_id', 'count'),
    pct_phaC=('has_phaC', lambda x: x.mean() * 100),
    pct_complete=('has_complete_pathway', lambda x: x.mean() * 100),
).round(1).sort_values('pct_phaC', ascending=False)

print('PHB prevalence by environment type:')
env_phb_stats

In [ ]:
# Statistical test: PHB enrichment in high-variability environments
known_var = env_phb[env_phb['primary_variability'].isin(['high', 'moderate', 'low'])]

# Chi-squared test: PHB status × variability
contingency = pd.crosstab(known_var['primary_variability'], known_var['has_phaC'])
chi2, p_val, dof, expected = stats.chi2_contingency(contingency)

print('Chi-squared test: PHB presence × environmental variability')
print(f'  chi2 = {chi2:.2f}, p = {p_val:.2e}, dof = {dof}')
print(f'\nContingency table:')
contingency

In [ ]:
# Figure: PHB prevalence by environment type
fig, ax = plt.subplots(figsize=(10, 6))

plot_data = env_phb_stats[env_phb_stats['n_species'] >= 20].sort_values('pct_phaC', ascending=True)

colors = []
for env in plot_data.index:
    var = VARIABILITY_SCORES.get(env, 'unknown')
    if var == 'high': colors.append('#F44336')
    elif var == 'moderate': colors.append('#FF9800')
    elif var == 'low': colors.append('#2196F3')
    else: colors.append('#9E9E9E')

ax.barh(range(len(plot_data)), plot_data['pct_phaC'], color=colors, alpha=0.8)
ax.set_yticks(range(len(plot_data)))
ax.set_yticklabels([f"{env} (n={int(n)})" for env, n in 
                     zip(plot_data.index, plot_data['n_species'])])
ax.set_xlabel('% of species with phaC')
ax.set_title('PHB Pathway Prevalence by Environment Type')

# Legend for variability colors
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#F44336', alpha=0.8, label='High variability'),
    Patch(facecolor='#FF9800', alpha=0.8, label='Moderate variability'),
    Patch(facecolor='#2196F3', alpha=0.8, label='Low variability'),
]
ax.legend(handles=legend_elements, loc='lower right', title='Expected temporal variability')

plt.tight_layout()
plt.savefig(os.path.join(FIG_DIR, 'phb_by_environment.png'), dpi=150, bbox_inches='tight')
plt.show()

## Part 2: AlphaEarth Embedding Analysis

Use 64-dim environmental embeddings to quantify the environmental breadth of PHB+ vs PHB- species.

**Note**: Only 28% of genomes (83K/293K) have embeddings. Environmental samples show 3.4x stronger geographic signal than human-associated samples.

In [ ]:
# Extract AlphaEarth embeddings
# Column names are A00-A63 (64-dimensional embedding)
emb_cols = [f'A{i:02d}' for i in range(64)]
emb_select = ', '.join([f'ae.{c}' for c in emb_cols])

embeddings = spark.sql(f"""
    SELECT g.genome_id, g.gtdb_species_clade_id,
           {emb_select}
    FROM kbase_ke_pangenome.genome g
    JOIN kbase_ke_pangenome.alphaearth_embeddings_all_years ae
        ON g.genome_id = ae.genome_id
""").toPandas()

print(f'Genomes with embeddings: {len(embeddings):,}')
print(f'Species with embeddings: {embeddings["gtdb_species_clade_id"].nunique():,}')

# Filter NaN embeddings (4.6% have NaN in some dimensions)
valid_mask = ~embeddings[emb_cols].isna().any(axis=1)
embeddings = embeddings[valid_mask]
print(f'After NaN filter: {len(embeddings):,} genomes')

In [ ]:
# Compute per-species embedding variance as a proxy for environmental breadth
# Species in more variable environments should have higher embedding variance

species_emb_stats = embeddings.groupby('gtdb_species_clade_id').agg(
    n_genomes_with_emb=('genome_id', 'count'),
    **{f'mean_{c}': (c, 'mean') for c in emb_cols},
    **{f'var_{c}': (c, 'var') for c in emb_cols},
).reset_index()

# Total variance across all 64 dimensions
var_cols = [f'var_{c}' for c in emb_cols]
species_emb_stats['total_emb_variance'] = species_emb_stats[var_cols].sum(axis=1)
species_emb_stats['mean_emb_variance'] = species_emb_stats[var_cols].mean(axis=1)

# Filter to species with >= 5 genomes for stable variance estimates
species_emb_stats = species_emb_stats[species_emb_stats['n_genomes_with_emb'] >= 5]
print(f'Species with >= 5 embedded genomes: {len(species_emb_stats):,}')

In [ ]:
# Merge embedding variance with PHB status
emb_phb = species_emb_stats[['gtdb_species_clade_id', 'n_genomes_with_emb', 
                              'total_emb_variance', 'mean_emb_variance']].merge(
    tax_phb[['gtdb_species_clade_id', 'phb_status', 'has_phaC', 'has_complete_pathway', 'gtdb_phylum']],
    on='gtdb_species_clade_id', how='inner'
)

print(f'Species with embeddings + PHB status: {len(emb_phb):,}')

# Compare embedding variance: PHB+ vs PHB-
phb_pos = emb_phb[emb_phb['has_phaC']]['total_emb_variance']
phb_neg = emb_phb[~emb_phb['has_phaC']]['total_emb_variance']

u_stat, p_mw = stats.mannwhitneyu(phb_pos, phb_neg, alternative='two-sided')

print(f'\nEmbedding variance (environmental breadth):')
print(f'  PHB+ (phaC present): median={phb_pos.median():.4f}, n={len(phb_pos):,}')
print(f'  PHB- (phaC absent):  median={phb_neg.median():.4f}, n={len(phb_neg):,}')
print(f'  Mann-Whitney U: U={u_stat:.0f}, p={p_mw:.2e}')

In [ ]:
# Figure: Embedding variance by PHB status
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Box plot
ax = axes[0]
emb_phb['PHB status'] = emb_phb['has_phaC'].map({True: 'phaC+', False: 'phaC-'})
sns.boxplot(data=emb_phb, x='PHB status', y='total_emb_variance', ax=ax,
            palette={'phaC+': '#4CAF50', 'phaC-': '#9E9E9E'})
ax.set_ylabel('Total embedding variance (environmental breadth)')
ax.set_title(f'Environmental Breadth: PHB+ vs PHB-\n(Mann-Whitney p={p_mw:.2e})')

# By pathway completeness
ax = axes[1]
status_order = ['complete', 'synthase_only', 'precursors_only', 'absent']
plot_df = emb_phb[emb_phb['phb_status'].isin(status_order)]
sns.boxplot(data=plot_df, x='phb_status', y='total_emb_variance', ax=ax,
            order=status_order)
ax.set_xlabel('PHB pathway status')
ax.set_ylabel('Total embedding variance')
ax.set_title('Environmental Breadth by Pathway Completeness')
ax.set_xticklabels([s.replace('_', '\n') for s in status_order])

plt.tight_layout()
plt.savefig(os.path.join(FIG_DIR, 'embedding_variance_phb.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Save results
env_phb.to_csv(os.path.join(DATA_DIR, 'species_environment.tsv'), sep='\t', index=False)
emb_phb.to_csv(os.path.join(DATA_DIR, 'phb_embedding_analysis.tsv'), sep='\t', index=False)
print(f'Saved environment data: {len(env_phb):,} species')
print(f'Saved embedding analysis: {len(emb_phb):,} species')

## Summary

### Key Findings (to be filled after execution)
- PHB prevalence by environment type: ?
- Chi-squared test result: ?
- AlphaEarth embedding variance PHB+ vs PHB-: ?
- Support for feast/famine hypothesis: ?

### Caveats
- AlphaEarth coverage is only 28%, biased toward clinical samples
- NCBI isolation_source is free text with inconsistent labeling
- Environment classification is a simplification — many genomes are from ambiguous sources
- Variability scoring is based on literature expectations, not direct measurements

### Next Notebook (NB04)
NMDC metagenomic analysis — test PHB enrichment across environments using independent data.